# Make Necessary Imports

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
import torch
import torch.nn as nn
import pandas as pd
import numpy as np
import re
import spacy
from collections import Counter
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import string
from torch.nn.utils.rnn import pack_padded_sequence, pad_packed_sequence
from sklearn.metrics import mean_squared_error
import random

# Read Dataset from CSV and 

In [ ]:
raw_data_path = '/content/BF_Titles_And_Classifications_LOCAdjusted_v2.csv'
destination_folder = '/content/drive/My Drive/lstm/Data'

In [ ]:
df_raw = pd.read_csv(raw_data_path)

# Initialize Necessary Encoding Functions


In [ ]:
def encode_sentence(text, vocab2index, N=70):
    tokenized = tokenize(text)
    encoded = np.zeros(N, dtype=int)
    enc1 = np.array([vocab2index.get(word, vocab2index["UNK"]) for word in tokenized])
    length = min(N, len(enc1))
    encoded[:length] = enc1[:length]
    return encoded, length

In [ ]:
tok = spacy.load('en')
def tokenize (text):
    text = re.sub(r"[^\x00-\x7F]+", " ", text)
    regex = re.compile('[' + re.escape(string.punctuation) + '0-9\\r\\t\\n]') # remove punctuation and numbers
    nopunct = regex.sub(" ", text.lower())
    return [token.text for token in tok.tokenizer(nopunct)]

# Clean up and encode dataset

## Create labels to encoding dictionary before working with data

In [ ]:
labels = pd.unique(df_raw["Call_Number"])
print(labels)
labels_dict = {}
for i in range(len(labels)):
  labels_dict[i] = labels[i]

['BF1-37:Psychology' 'BF38-64:Philosophy' 'BF65-172:Psychology'
 'BF173-175:Psychoanalysis' 'BF176:Pyshological tests and testing'
 'BF180-198:Experimental psychology' 'BF199-202:Psychology'
 'BF203:Gestalt psychology' 'BF204-206:Psychology'
 'BF207-209:Psychotropic drugs and other substances'
 'BF210-230:Psychology' 'BF231-299:Sensation. Aesthesiology'
 'BF300-308:Psychology' 'BF309-499:Consciousness. Congition'
 'BF501-505:Motivation' 'BF511-593:Affection. Feeling. Emotion'
 'BF594-607:Psychology' 'BF608-635:Will. Volition. Choice. Control'
 'BF636-637:Applied psychology'
 'BF638-648:New Thought. Menticulture, etc.'
 'BF660-685:Comparative psychology. Animal and human psychology'
 'BF692:Psychology of Sex. Sexual Behavior' 'BF693-696:Psychology'
 'BF697:Differential psychology. Individuality. Self' 'BF698:Personality'
 'BF699-711:Genetic Psychology' 'BF712-724:Developmental psychology'
 'BF725-727:Class psychology' 'BF728-794:Psychology'
 'BF795-839:Temperament. Character' 'BF840-885

In [ ]:
df_raw.Call_Number = pd.Categorical(df_raw.Call_Number) # Encode values of call numbers
df_raw['Call_Number'] = df_raw.Call_Number.cat.codes # Encode 
df_raw = df_raw.drop('ID', axis=1) # Drop ID column
df_raw['Title_length'] = df_raw['Title'].apply(lambda x: len(x.split()))  # Get length of each title

In [ ]:
mean_len = np.mean(df_raw['Title_length'])
print(mean_len)

8.494868686868687


In [ ]:
#count number of occurences of each word
counts = Counter()
for index, row in df_raw.iterrows():
    counts.update(tokenize(row['Title']))

In [ ]:
# deleting infrequent words
print("num_words before:",len(counts.keys()))
for word in list(counts):
    if counts[word] < 2:
        del counts[word]
print("num_words after:",len(counts.keys()))

num_words before: 21114
num_words after: 9570


In [ ]:
# creating vocabulary
vocab2index = {"":0, "UNK":1}
words = ["", "UNK"]
for word in counts:
    vocab2index[word] = len(words)
    words.append(word)

In [ ]:
df_raw['encoded'] = df_raw['Title'].apply(lambda x: np.array(encode_sentence(x,vocab2index )))
df_raw.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  """Entry point for launching an IPython kernel.


,Title,Call_Number,Title_length,encoded
0,advances in descriptive psychology official an...,0,13,"[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 4, 5, 0,..."
1,advances in descriptive psychology official an...,0,13,"[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 4, 5, 0,..."
2,advances in descriptive psychology official an...,0,13,"[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 4, 5, 0,..."
3,advances in descriptive psychology official an...,0,13,"[[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 4, 5, 0,..."
4,duke university psychological monographs,0,4,"[[13, 14, 15, 16, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0..."


In [ ]:
#check how balanced the dataset is
Counter(df_raw['Call_Number'])

Counter({0: 1265,
         1: 370,
         2: 253,
         3: 139,
         4: 32,
         5: 12,
         6: 500,
         7: 305,
         8: 126,
         9: 1,
         10: 129,
         11: 328,
         12: 431,
         13: 21,
         14: 283,
         15: 1741,
         16: 1,
         17: 186,
         18: 132,
         19: 107,
         20: 158,
         21: 75,
         22: 248,
         23: 167,
         24: 26,
         25: 136,
         26: 8,
         27: 5,
         28: 45,
         29: 4,
         30: 374,
         31: 1,
         32: 4316,
         33: 752,
         34: 117,
         35: 1719,
         36: 4,
         37: 204,
         38: 1938,
         39: 253,
         40: 2069,
         41: 318,
         42: 228,
         43: 2,
         44: 477,
         45: 1047,
         46: 158,
         47: 2852,
         48: 10,
         49: 398,
         50: 84,
         51: 106,
         52: 59,
         53: 30})

# Create Training and Test Sets from dataset (Dataset, Dataloader)


In [ ]:
X = list(df_raw['encoded'])
y = list(df_raw['Call_Number'])
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2)

In [ ]:
class ReviewsDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X
        self.y = Y
        
    def __len__(self):
        return len(self.y)
    
    def __getitem__(self, idx):
        return torch.from_numpy(self.X[idx][0].astype(np.int32)), self.y[idx], self.X[idx][1]

In [ ]:
train_ds = ReviewsDataset(X_train, y_train)
valid_ds = ReviewsDataset(X_valid, y_valid)

In [ ]:
batch_size = 1000
vocab_size = len(words)
train_dl = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
val_dl = DataLoader(valid_ds, batch_size=batch_size)

In [ ]:
num_outputs = len(labels)
print("Number of output classes: ", num_outputs)

Number of output classes:  54


# Model Set-Up Code



In [ ]:
def train_model(model, epochs=10, lr=0.001):
    parameters = filter(lambda p: p.requires_grad, model.parameters())
    optimizer = torch.optim.Adam(parameters, lr=lr)

    for i in range(epochs):
        model.train()
        sum_loss = 0.0
        total = 0
        for x, y, l in train_dl:
            x = x.long()
            y = y.long()

            y_pred = model(x, l)
            optimizer.zero_grad()
            loss = F.cross_entropy(y_pred, y)
            loss.backward()
            optimizer.step()
            sum_loss += loss.item()*y.shape[0]
            total += y.shape[0]
        val_loss, val_acc, val_rmse = validation_metrics(model, val_dl)
        print("Epoch# ", i)
        print("train loss %.3f, val loss %.3f, val accuracy %.3f, and val rmse %.3f" % (sum_loss/total, val_loss, val_acc, val_rmse))

def validation_metrics (model, valid_dl):
    model.eval()
    correct = 0
    total = 0
    sum_loss = 0.0
    sum_rmse = 0.0
    for x, y, l in valid_dl:
        x = x.long()
        y = y.long()
        y_hat = model(x, l)
        loss = F.cross_entropy(y_hat, y)
        pred = torch.max(y_hat, 1)[1]
        correct += (pred == y).float().sum()
        total += y.shape[0]
        sum_loss += loss.item()*y.shape[0]
        sum_rmse += np.sqrt(mean_squared_error(pred, y.unsqueeze(-1)))*y.shape[0]
    return sum_loss/total, correct/total, sum_rmse/total

In [ ]:
class LSTM_variable_input(torch.nn.Module) :
    def __init__(self, vocab_size, embedding_dim, hidden_dim) :
        super().__init__()
        self.hidden_dim = hidden_dim
        self.dropout = nn.Dropout(0.3)
        self.embeddings = nn.Embedding(vocab_size, embedding_dim, padding_idx=0)
        self.lstm = nn.LSTM(embedding_dim, hidden_dim, batch_first=True)
        self.linear = nn.Linear(hidden_dim, num_outputs)
        
    def forward(self, x, s):
        x = self.embeddings(x)
        x = self.dropout(x)
        x_pack = pack_padded_sequence(x, s, batch_first=True, enforce_sorted=False)
        out_pack, (ht, ct) = self.lstm(x_pack)
        out = self.linear(ht[-1])
        return out

# Initialize and Train Model

In [ ]:
import gc

if 'model' in locals():
  del model

model = LSTM_variable_input(vocab_size, 50, 50)

In [ ]:
train_model(model, epochs=30, lr=0.1)

Epoch#  0
train loss 3.047, val loss 2.693, val accuracy 0.288, and val rmse 14.912
Epoch#  1
train loss 2.502, val loss 2.420, val accuracy 0.379, and val rmse 15.205
Epoch#  2
train loss 2.209, val loss 2.256, val accuracy 0.420, and val rmse 14.462
Epoch#  3
train loss 2.037, val loss 2.233, val accuracy 0.441, and val rmse 14.716
Epoch#  4
train loss 1.928, val loss 2.160, val accuracy 0.462, and val rmse 13.772
Epoch#  5
train loss 1.840, val loss 2.110, val accuracy 0.479, and val rmse 14.492
Epoch#  6
train loss 1.782, val loss 2.080, val accuracy 0.484, and val rmse 13.897
Epoch#  7
train loss 1.718, val loss 2.052, val accuracy 0.492, and val rmse 14.090
Epoch#  8
train loss 1.680, val loss 2.031, val accuracy 0.498, and val rmse 13.987
Epoch#  9
train loss 1.658, val loss 2.066, val accuracy 0.500, and val rmse 13.624
Epoch#  10
train loss 1.629, val loss 2.043, val accuracy 0.501, and val rmse 13.870
Epoch#  11
train loss 1.610, val loss 2.053, val accuracy 0.509, and val rm

# Run a separate test on randomly selected values from the dataset

In the future, this should be run on test values from the e-book dataset.

In [ ]:
# Set number of random tests to run
test_size = 50

In [ ]:
# Get random sample of indicies based on size above

all_indicies = df_raw.index
all_indicies = all_indicies.tolist()
index_count = len(all_indicies)
random_test_indicies = random.sample(population=all_indicies, k=test_size)
print(random_test_indicies)

[15873, 3184, 17383, 13599, 19428, 11234, 24527, 7991, 5004, 9939, 12939, 17143, 21722, 6788, 24124, 7708, 11240, 11277, 19289, 14868, 14118, 12069, 1179, 336, 4142, 17687, 24423, 13941, 3851, 24022, 3667, 24132, 21778, 2950, 20089, 5197, 19072, 19941, 3906, 14859, 1873, 17548, 906, 11192, 19529, 3509, 9704, 13011, 22480, 12955]


In [ ]:
# Get the correct input output values from dataset

check_x = []
check_y = []

for index in random_test_indicies:
  check_x.append(df_raw['encoded'][index])
  check_y.append(df_raw['Call_Number'][index])

check_ds = ReviewsDataset(check_x, check_y)

In [ ]:
# Make Dataloader for random test values

batch_size = 1
vocab_size = len(words)
check_dl = DataLoader(check_ds, batch_size=batch_size, shuffle=True)

In [ ]:
model.eval()
num_correct = 0
total_num = test_size

for index, (x, y, l) in enumerate(check_dl):
  orig_title = df_raw['Title'][index]


  model_out = model(x, l)

  print("\n\nTest #", index)

  print("Original Book Title: ", orig_title)
  print("Expected book Classification: ", labels_dict[y.item()])
  print("Predicted Book Classification: ", labels_dict[torch.argmax(model_out).item()])

  if y.item() == torch.argmax(model_out).item():
    print("Model successfully identified correct category")
    num_correct += 1
  else:
    print("Incorrect labeling")




Test # 0
Original Book Title:  advances in descriptive psychology official annual publication of the society for descriptive psychology
Expected book Classification:  BF1730-1744:Occult sciences
Predicted Book Classification:  BF1730-1744:Occult sciences
Model successfully identified correct category


Test # 1
Original Book Title:  advances in descriptive psychology official annual publication of the society for descriptive psychology
Expected book Classification:  BF1048-1108:Hallucinations. Sleep. Dreaming. Visions
Predicted Book Classification:  BF1228-1389:Spiritualism
Incorrect labeling


Test # 2
Original Book Title:  advances in descriptive psychology official annual publication of the society for descriptive psychology
Expected book Classification:  BF1444-1486:Ghosts. Apparitions. Hauntings
Predicted Book Classification:  BF1444-1486:Ghosts. Apparitions. Hauntings
Model successfully identified correct category


Test # 3
Original Book Title:  advances in descriptive psychol

In [ ]:
print("FINAL RESULTS: ")
print("Accuracy = ", num_correct/total_num)

FINAL RESULTS: 
Accuracy =  0.54
